In [1]:
import numpy
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# load ascii text and covert to lowercase
filename = "/home/prateeksha/Desktop/wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [3]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab

Total Characters:  144424
Total Vocab:  45


In [5]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

Total Patterns:  144324


In [6]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [7]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 45)                11565     
Total params: 275,757
Trainable params: 275,757
Non-trainable params: 0
_________________________________________________________________


In [8]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [9]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
144324/144324 [==============================] - 1123s 8ms/step - loss: 2.9755
Epoch 2/20
144324/144324 [==============================] - 1037s 7ms/step - loss: 2.7655
Epoch 3/20
144324/144324 [==============================] - 967s 7ms/step - loss: 2.6641
Epoch 4/20
144324/144324 [==============================] - 969s 7ms/step - loss: 2.5826
Epoch 5/20
144324/144324 [==============================] - 970s 7ms/step - loss: 2.5216
Epoch 6/20
144324/144324 [==============================] - 971s 7ms/step - loss: 2.4663
Epoch 7/20
144324/144324 [==============================] - 972s 7ms/step - loss: 2.4192
Epoch 8/20
144324/144324 [==============================] - 971s 7ms/step - loss: 2.3795
Epoch 9/20
144324/144324 [==============================] - 971s 7ms/step - loss: 2.3383
Epoch 10/20
144324/144324 [==============================] - 973s 7ms/step - loss: 2.2980
Epoch 11/20
144324/144324 [==============================] - 973s 7ms/step - loss: 2.2570
Epoch 12/20
14432

In [12]:
filename =  'weights-improvement-19-1.9840.hdf5'
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer = 'adam')

In [13]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [16]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print "\nDone."

Seed:
"  said the caterpillar.

this was not an encouraging opening for a conversation. alice replied,
rathe "
r alo the woide thet she was aoling to tienting to tie thete was soenting and the harter whet she was not th theh  the was allie  to toe to her an tnee as the could. 
'the doessseen toue if ind toen i se got in,' said the monk turtle 
iitevery ao in spoke. 
'ieve you touldn'' said the manch hare.

'whet wer the woute gane to tee touting,' said the manch hare.

'whet wer the woute gane to tee touting,' said the manch hare.

'whet wer the woute gane to tee touting,' said the manch hare.

'whet wer the woute gane to tee touting,' said the manch hare.

'whet wer the woute gane to tee touting,' said the manch hare.

'whet wer the woute gane to tee touting,' said the manch hare.

'whet wer the woute gane to tee touting,' said the manch hare.

'whet wer the woute gane to tee touting,' said the manch hare.

'whet wer the woute gane to tee touting,' said the manch hare.

'whet wer the 

# larger LSTM

In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences =True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')


In [9]:
filepath = 'weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose =1, save_best_only = True, mode ='min' )
callbacks_list = [checkpoint]
model.fit(X, y, epochs=5, batch_size=64, callbacks= callbacks_list)

Epoch 1/5
144324/144324 [==============================] - 3419s 24ms/step - loss: 2.7899
Epoch 2/5
144324/144324 [==============================] - 3181s 22ms/step - loss: 2.4284
Epoch 3/5
144324/144324 [==============================] - 3176s 22ms/step - loss: 2.2130
Epoch 4/5
144324/144324 [==============================] - 3177s 22ms/step - loss: 2.0752
Epoch 5/5
144324/144324 [==============================] - 3177s 22ms/step - loss: 1.9783


In [10]:
filename =  'weights-improvement-05-1.9783-bigger.hdf5'
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer = 'adam')

In [11]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [13]:
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print "\nDone."

Seed:
"  then turned to the dormouse, and
repeated her question. 'why did they live at the bottom of a well? "
'

'i mnow the said the mittle saie to be a little shat the was a little shme the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a little was the was a lit